# Play

In [8]:
def match(agents, render=False, reset=True):
    if reset:
        obs, is_done = env.reset(), False    
    else:
        obs, is_done = env.regenerate_obs(), False    
    
    while not is_done:
        if agents[env.current_player] is None:
            state = env.get_state()
            prediction = calc_correct_output(env.players[env.current_player].hand_cards, env.table_card, env.players[env.current_player].tricks, env.players[1 - env.current_player].tricks)
           # print(prediction)
            env.set_state(state)
            #print("1", prediction, env.players[env.current_player].hand_cards[np.argmax(prediction)].color.name, env.players[env.current_player].hand_cards[np.argmax(prediction)].value.name)
            a = env.players[env.current_player].hand_cards[np.argmax(prediction)].id

            #break
        else:
            prediction = agents[env.current_player].predict_single(obs)[0]
           # print("2", prediction)
            #print(prediction, np.argmax(prediction))
            
            valid_moves = np.zeros((32,))       
            for card in env.players[env.current_player].hand_cards:
                valid_moves[card.id] = 1

            prediction_valid = np.ma.masked_where(valid_moves == 0, prediction)
            a = np.argmax(prediction_valid)            
            
                    
        obs, rew, is_done, _ = env.step(a)
        
        if render:
            env.render('human')
            sleep(1)
       # if env.lastTrick  is not None:
        #    break
    return env.current_player

def compare(agent1, agent2):
    agents = [agent1, agent2]
    first_player_wins = 0
    
    for i in range(10000):
        start_player = random.randint(0, 1)
        winner = match([agents[start_player], agents[1 - start_player]])
        first_player_wins += ((winner == 0) == (start_player == 0))
        #print(start_player, winner)
    
    return first_player_wins / 10000

def compare_given_games(agent1, agent2, games):
    agents = [agent1, agent2]
    first_player_wins = 0
    
    for game in games:
        for start_player in range(2):
            set_env(*game)
            env.current_player = start_player
            winner = match(agents, reset=False)
            first_player_wins += (winner == 0)
    
    return first_player_wins / (len(games) * 2)